In [2]:
import yfinance as yf
import pandas as pd
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt

In [ ]:
# --- Step 1: Data Retrieval ---
"""Gold, Silver, Crude Oil, Natural Gas, Copper, Energy, Materials, Financials,
    Industrials, Technology, Consumer Staples, Real Estate, Utilities, 
    Health Care, Consumer Discretionary, Communication Services (post-2018, limited history)"""

tickers = ["GC=F", "SI=F", "CL=F", "NG=F", "HG=F", "XLE", "XLB", "XLF",
           "XLI", "XLK", "XLP", "XLRE", "XLU", "XLV", "XLY", "XLC"]

prices = yf.download(tickers, start="1900-01-01", auto_adjust=True)["Adj Close"]